In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/혼공/ANN/Churn_Modelling.csv')

X = data.iloc[:, 3:-1].values
y = data.iloc[:, -1].values

le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = ct.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

ann = keras.models.Sequential()
ann.add(keras.layers.Dense(10, activation='relu'))
ann.add(keras.layers.Dense(10, activation='relu'))
ann.add(keras.layers.Dropout(0.3))
ann.add(keras.layers.Dense(1, activation='sigmoid'))
ann.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

check_cb = keras.callbacks.ModelCheckpoint('test.h5', save_best_only=True)
early_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

ann.fit(X_train, y_train, epochs=100, verbose=1, validation_data=(X_test, y_test), callbacks=[check_cb, early_cb])
#verbose=1: 진행 막대 표시
ann.summary()

Epoch 1/100
250/250 [==============================] - 5s 13ms/step - loss: 0.5624 - accuracy: 0.7434 - val_loss: 0.4466 - val_accuracy: 0.8145
Epoch 2/100
  6/250 [..............................] - ETA: 3s - loss: 0.5285 - accuracy: 0.7656

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


250/250 [==============================] - 2s 10ms/step - loss: 0.4820 - accuracy: 0.7916 - val_loss: 0.4191 - val_accuracy: 0.8145
Epoch 3/100
250/250 [==============================] - 2s 6ms/step - loss: 0.4587 - accuracy: 0.7916 - val_loss: 0.4107 - val_accuracy: 0.8145
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4521 - accuracy: 0.7920 - val_loss: 0.4076 - val_accuracy: 0.8165
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4485 - accuracy: 0.7977 - val_loss: 0.4048 - val_accuracy: 0.8330
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4389 - accuracy: 0.8065 - val_loss: 0.3984 - val_accuracy: 0.8370
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4350 - accuracy: 0.8149 - val_loss: 0.3948 - val_accuracy: 0.8410
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4290 - accuracy: 0.8186 - val_loss: 0.3916 - val_accuracy: 0.8405
Epoch 9/10

In [ ]:
a = ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5
b = ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))
print(a)
print(b)
#0 -> 계속 이용 / 1 -> 떠남

pred = ann.predict(X_test)
pred = (pred > 0.5)
print(np.concatenate((pred.reshape(len(pred), 1), y_test.reshape(len(y_test), 1)), 1))

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, pred)
print(cm)

accuracy_score(y_test, pred) #정확도 89%

#(1)이용 유지 1519명 예측
#(4)이용 취소 212명 예측
#(2)이용 취소 예측 76명 틀림
#(3)이용 유지 193명 틀림

1/1 [==============================] - 0s 22ms/step
[[False]]
[[0.03657942]]
63/63 [==============================] - 0s 2ms/step
[[0 0]
 [0 0]
 [1 1]
 ...
 [0 0]
 [0 0]
 [1 0]]
[[1568   61]
 [ 196  175]]


0.8715